In [219]:
from textblob.classifiers import NLTKClassifier,NaiveBayesClassifier,DecisionTreeClassifier,PositiveNaiveBayesClassifier,MaxEntClassifier
import pandas as pd
import re

path = '/Users/Jared/Documents/CSCE489/489Project/Turk/'

In [220]:
turk_df = pd.read_csv(path + 'Batch_2606965_batch_results.csv') 
turk_df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.status_id,Input.content,Answer.sentiment,Approve,Reject
0,301KG0KX9CFC0MHLW3ECT80VTS3H2D,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:16 PST 2016,3,BatchId:2606965;,172800,...,NaN,3,100% (143/143),100% (143/143),100% (143/143),13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive,NaN,NaN
1,301KG0KX9CFC0MHLW3ECT80VTS3H2D,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:16 PST 2016,3,BatchId:2606965;,172800,...,NaN,3,100% (349/349),100% (349/349),100% (349/349),13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive,NaN,NaN
2,301KG0KX9CFC0MHLW3ECT80VTS3H2D,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:16 PST 2016,3,BatchId:2606965;,172800,...,NaN,7,100% (132/132),100% (132/132),100% (132/132),13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Neutral,NaN,NaN
3,301KG0KX9CFC0MHLW3ECT80VTS42HZ,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:36 PST 2016,3,BatchId:2606965;,172800,...,NaN,51,100% (147/147),100% (147/147),100% (147/147),131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Positive,NaN,NaN
4,301KG0KX9CFC0MHLW3ECT80VTS42HZ,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:36 PST 2016,3,BatchId:2606965;,172800,...,NaN,3,100% (99/99),100% (99/99),100% (99/99),131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Neutral,NaN,NaN


In [221]:
relevant_df = turk_df[['Input.status_id','Input.content','Answer.sentiment']]
relevant_df.columns = ['status_id','content','sentiment']
relevant_df.head()

,status_id,content,sentiment
0,13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive
1,13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive
2,13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Neutral
3,131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Positive
4,131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Neutral


In [222]:
df1 = relevant_df[['status_id','content']]
df2 = relevant_df[['status_id','sentiment']]

In [223]:
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]',' ', text)

In [224]:
df1 = df1.groupby(df1.status_id).first()
#df1.index.name = None
df1.head()

,content
status_id,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ..."
131459315949_10152753624975950,Second fiddle to Hillary Clinton in the debate...
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...


In [225]:
df1['content'] = df1.content.apply(remove_non_ascii)
df1.head()

,content
status_id,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ..."
131459315949_10152753624975950,Second fiddle to Hillary Clinton in the debate...
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...


In [226]:
def choose_sentiment(label):
    if label.count('Positive') >= 2:
        return 'Positive'
    elif label.count('Neutral') >= 2:
        return 'Neutral'
    elif label.count('Negative') >= 2:
        return 'Negative'
    else:
        return 'Inconclusive'

In [227]:
df2 = df2.groupby(df2.status_id).sum()
df2.head()

,sentiment
status_id,
131459315949_10152751596575950,NeutralNegativeNegative
131459315949_10152753624975950,NeutralPositiveNegative
131459315949_10152754227800950,PositiveNeutralNeutral
131459315949_10152758428025950,NegativeNegativeNeutral
131459315949_10152766742570950,NeutralNegativeNeutral


In [228]:
df2['sentiment'] = df2.sentiment.apply(choose_sentiment)
df2.head()

,sentiment
status_id,
131459315949_10152751596575950,Negative
131459315949_10152753624975950,Inconclusive
131459315949_10152754227800950,Neutral
131459315949_10152758428025950,Negative
131459315949_10152766742570950,Neutral


In [229]:
df2 = df2[df2.sentiment != 'Inconclusive']
df2.head()

,sentiment
status_id,
131459315949_10152751596575950,Negative
131459315949_10152754227800950,Neutral
131459315949_10152758428025950,Negative
131459315949_10152766742570950,Neutral
131459315949_10152791196665950,Positive


In [230]:
merged_df = df1.merge(df2,how='right',left_index=True,right_index=True)
merged_df.head()

,content,sentiment
status_id,,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ...",Negative
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...,Neutral
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...,Negative
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...,Neutral
131459315949_10152791196665950,A look at 5 crucial issues in the Canadian ele...,Positive


In [231]:
tuple_arr = list(zip(merged_df.content,merged_df.sentiment))
size = len(tuple_arr)
halfway_pt = size/2

print size

train = tuple_arr[:halfway_pt]
test = tuple_arr[halfway_pt:]
print len(train)
print len(test)

847
423
424


In [232]:
nltk_classifier = NLTKClassifier(train)
print 'Done Training NLTK Classifier!'

Done Training NLTK Classifier!


In [233]:
naivebayes_classifier = NaiveBayesClassifier(train)
print 'Done Training Naive Bayes Classifier!'

Done Training Naive Bayes Classifier!


In [234]:
decisiontree_classifier = DecisionTreeClassifier(train)
print 'Done Training Decision Tree Classifier!'

Done Training Decision Tree Classifier!


In [235]:
positivenaivebayes_classifier = PositiveNaiveBayesClassifier(train)
print 'Done Training Positive Naive Bayes Classifier!'

TypeError: __init__() takes at least 3 arguments (2 given)

In [236]:
maxent_classifier = MaxEntClassifier(train)
print 'Done Training Max Ent Classifier!'

Done Training Max Ent Classifier!


In [239]:
#nltk_classifier_accuracy = nltk_classifier.accuracy(test)
naivebayes_classifier_accuracy = naivebayes_classifier.accuracy(test)
decisiontree_classifier_accuracy = decisiontree_classifier.accuracy(test)
#positivenaivebayes_classifier_accuracy = positivenaivebayes_classifier.accuracy(test)
maxent_classifier_accuracy = maxent_classifier.accuracy(test)

#print "NLTK Accuracy: ", nltk_classifier_accuracy
print "Naive Bayes Accuracy: ", naivebayes_classifier_accuracy
print "Decision Tree Accuracy: ", decisiontree_classifier_accuracy
#print "Positive Naive Bayes Accuracy: ", positivenaivebayes_classifier_accuracy
print "Maxent Accuracy: ", maxent_classifier_accuracy

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.314
         Final               nan        0.314
Naive Bayes Accuracy:  0.452830188679
Decision Tree Accuracy:  0.408018867925
Maxent Accuracy:  0.301886792453
